In [1]:
%run database.py
import pandas as pd

In [2]:
conn = create_connection()

Connection to SQLite DB successful


In [3]:
# define columns as a list of pairs with title and type
columns = [
    ('HiddenService', 'TEXT PRIMARY KEY'), # TODO: Make FK in other tables?
]

In [4]:
#drop the table from the connection
drop_table(services_tablename, conn)

Table services dropped


In [5]:
conn.execute(
    generateCreateTableQuery(services_tablename, columns)
)

CREATE TABLE IF NOT EXISTS "services" (
"HiddenService" TEXT PRIMARY KEY);


In [7]:
# get all the hidden services from the table pings and store them into a dataframe
df = pd.read_sql_query("""
                       SELECT 
                       HiddenService,
                       MIN(LastContactedDate) as FirstSeen_Date,
                       MAX(LastUpDate) as LastSeen_Date
                       FROM pings
                       GROUP BY HiddenService
                       """, conn)
# make HiddenService the index
df.set_index('HiddenService', inplace=True)

# convert date strings to datetime objects
df['FirstSeen_Date'] = pd.to_datetime(df['FirstSeen_Date'])
df['LastSeen_Date'] = pd.to_datetime(df['LastSeen_Date'])

# add age column
df['Age'] = df['LastSeen_Date'] - df['FirstSeen_Date']

In [9]:
# query the earliest and latest title per hidden service based on the LastContactedDate
df2 = pd.read_sql_query("""
                        SELECT
                            HiddenService,
                            Title as Title_Earliest,
                            Language as Language_Earliest,
                            LastContactedDate
                        FROM pings
                        WHERE LastContactedDate IN (
                            SELECT MIN(LastContactedDate)
                            FROM pings
                            GROUP BY HiddenService
                        )
                        ;   
                        """, conn)

In [14]:
# query the most frequent language per hidden service based on the count of the language in pings
df3 = pd.read_sql_query("""
                        WITH LanguageCounts AS (
                            -- Calculate the count of each language for each hidden service.
                            SELECT
                                HiddenService,
                                Language,
                                COUNT(*) AS LangCount
                            FROM pings
                            GROUP BY HiddenService, Language
                        ),

                        MaxLanguageCounts AS (
                            -- Determine the maximum count for each hidden service.
                            SELECT
                                HiddenService,
                                MAX(LangCount) AS MaxLangCount
                            FROM LanguageCounts
                            GROUP BY HiddenService
                        )

                        -- Join the two CTEs to determine the most frequent language for each hidden service.
                        SELECT
                            lc.HiddenService,
                            lc.Language AS Language_MostFrequent
                        FROM LanguageCounts lc
                        JOIN MaxLanguageCounts mlc
                        ON lc.HiddenService = mlc.HiddenService AND lc.LangCount = mlc.MaxLangCount
                        """, conn)

In [16]:
df2

,HiddenService,Title_Earliest,Language_Earliest,LastContactedDate
0,484de3c52dfbd2e81744a9037ac3a9554fc443d9eec1a9...,403 Forbidden,da,2018-02-15
1,84eed506a2344a62f791d9e80380a8ac309f9586954edb...,OnionDrop,en,2018-02-15
2,f3e537fb895fb820d5c8862ce40146a5b3f1d25ea743c6...,Sign In - Hidden Clubs,en,2018-02-15
3,a04854eecb5879d3b6d84e95297f72762c50e6396fc4f6...,None,None,2018-02-15
4,83ea422f63436bf945d7d39d29bda3859faaca78841ba6...,None,None,2018-02-15
...,...,...,...,...
16321647,10040ce4c3fb72fcaee32b658f31670c24244e1a484b75...,Tor Blog | The Tor Project,en,2022-10-03
16321648,ce44c3ad63ba4263017199f400e942d8e6f16cdf2f04a9...,"Technology, taught collectively. — Tech Learni...",en,2022-10-03
16321649,eb863fc7bb8f8e9cc2e0351b28cf1b0926fd5ee4c71d0e...,MARKET,en,2022-10-03
16321650,aebc9e8d456f0f18acdd94fca2948eddfa8edf22768421...,LoliPorn,en,2022-10-03


In [17]:
#combine the dataframes
df = df.join(df2.set_index('HiddenService'))
df = df.join(df3.set_index('HiddenService'))
df

MemoryError: Unable to allocate 39.5 GiB for an array with shape (5301583616,) and data type int64

In [20]:
# alter the table to add the new columns
conn.execute("""
                ALTER TABLE """ + services_tablename + """
                ADD COLUMN FirstSeen_Date TEXT
            """)

OperationalError: duplicate column name: FirstSeen_Date

In [26]:
conn.execute("""
    UPDATE """ + services_tablename + """ AS s
    SET FirstSeen_Date = (
        SELECT 
            MAX(LastContactedDate) as FirstSeen_Date
        FROM """ + pings_tablename + """ as p
        WHERE p.HiddenService = s.HiddenService
    );
""")
conn.commit()

In [ ]:
WITH LanguageCounts AS (
    SELECT 
        HiddenService,
        Language,
        COUNT(Language) AS lang_count
    FROM pings
    GROUP BY HiddenService, Language
),

FirstLanguage AS (
    SELECT 
        HiddenService,
        FIRST_VALUE(Language) OVER (PARTITION BY HiddenService ORDER BY lang_count DESC) AS most_used_language
    FROM LanguageCounts
)

SELECT DISTINCT
    HiddenService,
    most_used_language
    
FROM FirstLanguage;

In [20]:
df

,FirstSeen_Date,LastSeen_Date
HiddenService,,
00003ac5d174c575728e87444efda3637ae6c7ad34cc4e77bcb10bd450c17e89,2017-09-15,None
0000589b844ab7a343596717fa781540a554973210fe47972ee5d7e4b9a322df,2017-11-26,None
00008a4a2b07f5e24b2faa42f20b20b0c269ef2bfea1d8066be5b0828720bc93,2017-09-27,None
0000e8523baebd71a193279a082929f54d3bb944986a9ebc344a721d207957e2,2017-11-09,None
00010bd1b2520e9f07a9b96bbb533b542e9b2ca13e44c76acffdb9871da08c16,2017-09-21,None
...,...,...
ffff7ef98c91f41c50c4b1e1be28b24f9e99a17a9e636b6c00eb949fe6a209f7,2017-11-10,2020-04-28
ffffa0a6ac2431cae1b7172820675f2f065dafd1a7caae813b773c72b8196255,2017-09-28,None
ffffc3a8ea15d2e04002b9c80954d69835bb59f37066527460d01e51faa187fe,2017-10-23,None


In [9]:
# insert the hidden services into the table services
# don't insert the rowid
df.to_sql(services_tablename, conn, if_exists='append', index=True, index_label='HiddenService')

411580

In [10]:
#!pip install sqlite-utils
!sqlite-utils add-foreign-key darkasette.db pings HiddenService services HiddenService 

In [3]:
close_connection(conn)

The SQLite connection is closed


In [22]:
conn.close()